In [1]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print(type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


In [3]:
def Plural(palavra):
    """Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
        
    Returns:
        str: A string with 's' added to it.
    """
    return '{}s'.format(palavra)

print(Plural('gato'))

gatos


In [4]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
        
    Returns:
        str: A string with 's' added to it.



In [5]:
assert Plural('rato') == 'ratos', 'resultado incorreto!'
print('OK')

OK


In [8]:
pluralRDD = palavrasRDD.map(Plural)
print(pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [10]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'],'valores incorretos!'
print('OK')

OK


In [11]:
pluralLambdaRDD = palavrasRDD.map(lambda x:'{}s'.format(x))
print(pluralLambdaRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [12]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'],'valores incorretos!'
print('OK')

OK


In [13]:
pluralTamanho = (pluralRDD.map(lambda x:len(x)).collect())
print(pluralTamanho)

[5, 9, 5, 5, 5]


In [14]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print ("OK")

OK


In [15]:
palavraPar = palavrasRDD.map(lambda x:(x,1))
print(palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [16]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print ("OK")

OK


In [25]:
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    valores = list(valor)
    print('{}: {}'.format(chave, valores))

rato: [1, 1]
elefante: [1]
gato: [1, 1]


In [26]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])], 'Valores incorretos!'
print ("OK")

OK


In [49]:
contagemGroup = palavrasGrupo.map(lambda x:(x[0],sum(x[1])))
print(contagemGroup.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [50]:
assert sorted(contagemGroup.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [95]:
contagem = palavraPar.reduceByKey(lambda x,y:x+y)
print(contagem.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [96]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [97]:
contagemFinal = (palavrasRDD
                 .map(lambda x:(x,1))
                 .reduceByKey(lambda x,y:x+y))
print(contagemFinal.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [98]:
assert sorted(contagemFinal.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [99]:
palavrasUnicas = (contagemFinal
                  .count()
                 )
print(palavrasUnicas)

3


In [100]:
assert palavrasUnicas==3, 'valor incorreto!'
print ("OK")

OK


In [103]:
total = (contagemFinal
        .map(lambda x:x[1])
        .reduce(lambda x,y:x+y)
        )
media = total / float(palavrasUnicas)
print(total)
print(round(media, 2))

5
1.67


In [104]:
assert round(media, 2)==1.67, 'valores incorretos!'
print ("OK")

OK


In [105]:
def contaPalavras(chavesRDD):
    """Creates a pair RDD witht word counts from an RDD of words.
    
    Args:
        chavesRDD (RDD of str): An RDD consisting of words.
    
    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    return (chavesRDD
           .map(lambda x:(x,1))
           .reduceByKey(lambda x,y:x+y)
           )

print(contaPalavras(palavrasRDD).collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [106]:
assert sorted(contaPalavras(palavrasRDD).collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [107]:
import re
def removerPontuacao(texto):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        texto (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower()
print (removerPontuacao('Ola, quem esta ai??!'))
print (removerPontuacao(' Sem espaco e_sublinhado!'))


ola quem esta ai
sem espaco esublinhado


In [108]:
assert removerPontuacao(' O uso de virgulas, embora permitido, nao deve contar. ')=='o uso de virgulas embora permitido nao deve contar', 'string incorreta!'
print ("OK")

OK


In [111]:
import os.path

arquivo = os.path.join('', 'pg100.txt')

shakesRDD = (sc
             .textFile(arquivo, 8)
             .map(removerPontuacao)
            )

print('\n'.join(shakesRDD
               .zipWithIndex()
               .map(lambda linha: '{0}: {1}'.format(linha[0], linha[1]))
               .take(15)
               )
     )

: 0
project gutenbergs the complete works of william shakespeare by william: 1
shakespeare: 2
: 3
this ebook is for the use of anyone anywhere in the united states and: 4
most other parts of the world at no cost and with almost no restrictions: 5
whatsoever  you may copy it give it away or reuse it under the terms: 6
of the project gutenberg license included with this ebook or online at: 7
wwwgutenbergorg  if you are not located in the united states youll: 8
have to check the laws of the country where you are located before using: 9
this ebook: 10
: 11
see at the end of this file  content note added in 2017: 12
: 13
: 14


In [114]:
shakesPalavrasRDD = shakesRDD.map(lambda x:x.split(' '))
total = shakesPalavrasRDD.count()
print(shakesPalavrasRDD.take(5))
print(total)

[[u''], [u'project', u'gutenbergs', u'the', u'complete', u'works', u'of', u'william', u'shakespeare', u'by', u'william'], [u'shakespeare'], [u''], [u'this', u'ebook', u'is', u'for', u'the', u'use', u'of', u'anyone', u'anywhere', u'in', u'the', u'united', u'states', u'and']]
147929


In [115]:
shakesPalavrasRDD = shakesRDD.flatMap(lambda x: x.split())
total = shakesPalavrasRDD.count()
print(shakesPalavrasRDD.take(5))
print(total)

[u'project', u'gutenbergs', u'the', u'complete', u'works']
959359


In [116]:
assert total==959359, "valor incorreto de palavras!"
print ("OK")
assert shakesPalavrasRDD.take(5)==['project', 'gutenbergs', 'the', 'complete', 'works'],'lista incorreta de palavras'
print ("OK")

OK
OK


In [123]:
top15 = contaPalavras(shakesPalavrasRDD).takeOrdered(15, key = lambda x:-x[1])
print('\n'.join(map(lambda x: '{}: {}'.format(x[0], x[1]), top15)))


the: 29996
and: 28353
i: 21860
to: 20885
of: 18811
a: 15992
you: 14439
my: 13191
in: 12027
that: 11782
is: 9711
not: 9068
with: 8521
me: 8271
for: 8184


In [124]:
assert top15 == [('the', 29996), ('and', 28353), ('i', 21860), ('to', 20885), ('of', 18811), ('a', 15992), ('you', 14439), ('my', 13191), ('in', 12027), ('that', 11782), ('is', 9711), ('not', 9068), ('with', 8521), ('me', 8271), ('for', 8184)],'valores incorretos!'
print ("OK")

OK


In [125]:
import numpy as np

def pNorm(p):
    """Generates a function to calculate the p-Norm between two points.
    
    Args:
        p (int): The integer p
        
    Returns:
        Dist: A function that calculates the p-Norm.
    """
    
    def Dist(x, y):
        return np.power(np.power(np.abs(x - y), p).sum(), 1/float(p))
    return Dist

In [126]:
np.random.seed(42)
numPointsRDD = sc.parallelize(enumerate(np.random.random(size=(10,100))))

In [138]:
cartPointsRDD = numPointsRDD.cartesian(numPointsRDD)

In [139]:
cartPointsRDD.take(1)

[((0, array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
          0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258,
          0.02058449, 0.96990985, 0.83244264, 0.21233911, 0.18182497,
          0.18340451, 0.30424224, 0.52475643, 0.43194502, 0.29122914,
          0.61185289, 0.13949386, 0.29214465, 0.36636184, 0.45606998,
          0.78517596, 0.19967378, 0.51423444, 0.59241457, 0.04645041,
          0.60754485, 0.17052412, 0.06505159, 0.94888554, 0.96563203,
          0.80839735, 0.30461377, 0.09767211, 0.68423303, 0.44015249,
          0.12203823, 0.49517691, 0.03438852, 0.9093204 , 0.25877998,
          0.66252228, 0.31171108, 0.52006802, 0.54671028, 0.18485446,
          0.96958463, 0.77513282, 0.93949894, 0.89482735, 0.59789998,
          0.92187424, 0.0884925 , 0.19598286, 0.04522729, 0.32533033,
          0.38867729, 0.27134903, 0.82873751, 0.35675333, 0.28093451,
          0.54269608, 0.14092422, 0.80219698, 0.07455064, 0.98688694,
          0.77224

In [142]:
cartPointsParesRDD = cartPointsRDD.map(lambda x:((x[0][0],x[1][0]),(x[0][1],x[1][1])))

In [143]:
cartPointsParesRDD.take(1)

[((0, 0), (array([0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
          0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258,
          0.02058449, 0.96990985, 0.83244264, 0.21233911, 0.18182497,
          0.18340451, 0.30424224, 0.52475643, 0.43194502, 0.29122914,
          0.61185289, 0.13949386, 0.29214465, 0.36636184, 0.45606998,
          0.78517596, 0.19967378, 0.51423444, 0.59241457, 0.04645041,
          0.60754485, 0.17052412, 0.06505159, 0.94888554, 0.96563203,
          0.80839735, 0.30461377, 0.09767211, 0.68423303, 0.44015249,
          0.12203823, 0.49517691, 0.03438852, 0.9093204 , 0.25877998,
          0.66252228, 0.31171108, 0.52006802, 0.54671028, 0.18485446,
          0.96958463, 0.77513282, 0.93949894, 0.89482735, 0.59789998,
          0.92187424, 0.0884925 , 0.19598286, 0.04522729, 0.32533033,
          0.38867729, 0.27134903, 0.82873751, 0.35675333, 0.28093451,
          0.54269608, 0.14092422, 0.80219698, 0.07455064, 0.98688694,
          0.

In [162]:
Euclid = pNorm(2)

In [174]:
distRDD = cartPointsParesRDD.map(lambda x:(x[0], Euclid(x[1][0],x[1][1])))

In [178]:
statRDD = distRDD.map(lambda x: x[1])

In [179]:
minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()

In [180]:
print(minv, maxv, meanv)

(0.0, 4.709048183663605, 3.7511916889753705)


In [181]:
assert (minv.round(2), maxv.round(2), meanv.round(2))==(0.0, 4.71, 3.75), 'Valores incorretos'
print ("OK")

OK
